In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("BizAnalytix") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

scSpark = configure_spark_with_delta_pip(builder).getOrCreate()
people_path = "data/people.csv"

people_data = scSpark.read.csv(people_path, header='true', sep=',', inferSchema=False)

22/06/22 14:52:14 WARN Utils: Your hostname, Jarvis resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlo1)
22/06/22 14:52:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/sid/Documents/Assessment/bizanalytix/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sid/.ivy2/cache
The jars for the packages stored in: /home/sid/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c26800c6-d9f5-4669-840f-e4def270b372;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 205ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted||

In [2]:
people_data.printSchema()

root
 |-- person_ID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- middle: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)
 |-- title: string (nullable = true)



In [3]:
people_data.limit(10).toPandas()

,person_ID,name,first,last,middle,email,phone,fax,title
0,3130,"Burks, Rosella",Rosella,Burks,None,BurksR@univ.edu,963.555.1253,963.777.4065,Professor
1,3297,"Avila, Damien",Damien,Avila,None,AvilaD@univ.edu,963.555.1352,963.777.7914,Professor
2,3547,"Olsen, Robin",Robin,Olsen,None,OlsenR@univ.edu,963.555.1378,963.777.9262,Assistant Professor
3,1538,"Moises, Edgar Estes",Edgar,Moises,Estes,MoisesE@univ.edu,963.555.2731x3565,963.777.8264,Professor
4,2941,"Brian, Heath Pruitt",Heath,Brian,Pruitt,BrianH@univ.edu,963.555.2800,963.777.7249,Associate Curator
5,2401,"Claude, Elvin Haney",Elvin,Claude,Haney,ClaudeE@univ.edu,963.555.2902,963.777.9730,Curator
6,2070,"Mosley, Edmund",Edmund,Mosley,None,MosleyE@univ.edu,963.555.2945,963.777.9285,Assistant Professor
7,2561,"Derek, Antoine Mccoy",Antoine,Derek,Mccoy,DerekA@univ.edu,963.555.2992,963.777.5454,Curator
8,1625,"Hawkins, Callie",Callie,Hawkins,None,HawkinsC@univ.edu,963.555.3350x6480,963.777.4949,Professor
9,1307,"Pate, Andrea",Andrea,Pate,None,PateA@univ.edu,963.555.3723,963.777.5049,Professor


In [4]:
people_data.write.format("delta").saveAsTable("default.people_delta")

In [5]:
df = scSpark.read.format("delta").load("spark-warehouse/people_delta")  # query table by path

In [7]:
# extracting number of rows from the Dataframe
row = df.count()
   
# extracting number of columns from the Dataframe
col = len(df.columns)

# printing
print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Records/Rows are: {row}')
print(f'Number of Columns are: {col}')

Dimension of the Dataframe is: (40, 9)
Number of Records/Rows are: 40
Number of Columns are: 9
